In [1]:
from IPython.display import display, HTML

def show_df(df):
    display(df.shape)
    display(df.head())

In [2]:
import pandas as pd
raw_dataset = pd.read_csv("FinalProject1_dataset.csv")
show_df(raw_dataset)
display(raw_dataset["direction"].value_counts())

raw_dataset = raw_dataset[raw_dataset.direction != 'UNKNOWN']
#raw_dataset.direction["FROM_INSIDE_TO_OUTSIDE"] = 0
#raw_dataset.direction["FROM_OUTSIDE_TO_INSIDE"] = 1
raw_dataset = raw_dataset.replace("FROM_INSIDE_TO_OUTSIDE", 0)
raw_dataset = raw_dataset.replace("FROM_OUTSIDE_TO_INSIDE", 1)
display(raw_dataset["direction"].value_counts())

(9984, 7)

,_id,name,locationStatus,direction,timestamp,rssiOne,rssiTwo
0,4744423E612A1ED103904DB4F5987BA6,FF:FF:C2:1D:9B:54,OUTSIDE,FROM_INSIDE_TO_OUTSIDE,1551367500637,-79,-90
1,C245BE3DABC6B3B534FD3D0E660049B2,FF:FF:C2:1D:9B:54,OUTSIDE,FROM_INSIDE_TO_OUTSIDE,1551367501323,-81,-90
2,854BB66FF73F63F0FB15149B80434D80,FF:FF:C2:1D:9B:54,OUTSIDE,FROM_INSIDE_TO_OUTSIDE,1551367502416,-83,-91
3,1E448462718ADB1DD73E9553A7B268B7,FF:FF:C2:1D:9B:54,OUTSIDE,FROM_INSIDE_TO_OUTSIDE,1551367504813,-80,-91
4,0D4BAE97FE52DBAF3E596C72CC45C78F,FF:FF:C2:1D:9B:54,OUTSIDE,FROM_INSIDE_TO_OUTSIDE,1551367504813,-80,-91


FROM_INSIDE_TO_OUTSIDE    4540
FROM_OUTSIDE_TO_INSIDE    4178
UNKNOWN                   1266
Name: direction, dtype: int64

0    4540
1    4178
Name: direction, dtype: int64

In [3]:
clean_dataset = raw_dataset.drop(["_id"], axis=1)
show_df(clean_dataset)

(8718, 6)

,name,locationStatus,direction,timestamp,rssiOne,rssiTwo
0,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367500637,-79,-90
1,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367501323,-81,-90
2,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367502416,-83,-91
3,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367504813,-80,-91
4,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367504813,-80,-91


In [4]:
import numpy as np
from sklearn.model_selection import train_test_split

x = clean_dataset.drop(["locationStatus", "timestamp", "name"], axis=1)
y = clean_dataset["locationStatus"]

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.3, stratify=y)
X_train.head()

,direction,rssiOne,rssiTwo
837,0,-81,-89
6388,0,-85,-74
5979,0,-83,-83
9137,1,-102,-91
6186,1,-90,-84


In [5]:
import matplotlib.pyplot as plt

raw_dataset["rssiOne"]
plt.plot(raw_dataset["timestamp"], raw_dataset["rssiOne"])

# Naive Bayes

In [6]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[437 133 165]
 [109 723 141]
 [122 125 661]]
              precision    recall  f1-score   support

      INSIDE       0.65      0.59      0.62       735
IN_VESTIBULE       0.74      0.74      0.74       973
     OUTSIDE       0.68      0.73      0.71       908

   micro avg       0.70      0.70      0.70      2616
   macro avg       0.69      0.69      0.69      2616
weighted avg       0.70      0.70      0.69      2616

0.6961009174311926


# Decision Tree (CART)

In [7]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
y_pred = clf.fit(X_train, y_train).predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[496 150  89]
 [174 657 142]
 [101 134 673]]
              precision    recall  f1-score   support

      INSIDE       0.64      0.67      0.66       735
IN_VESTIBULE       0.70      0.68      0.69       973
     OUTSIDE       0.74      0.74      0.74       908

   micro avg       0.70      0.70      0.70      2616
   macro avg       0.70      0.70      0.70      2616
weighted avg       0.70      0.70      0.70      2616

0.6980122324159022


# SVM

In [8]:
from sklearn.svm import SVC
# fit a SVM model to the data
svm = SVC()
y_pred = svm.fit(X_train, y_train).predict(X_test)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

C:\anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[[490 147  98]
 [104 720 149]
 [ 81  98 729]]
              precision    recall  f1-score   support

      INSIDE       0.73      0.67      0.70       735
IN_VESTIBULE       0.75      0.74      0.74       973
     OUTSIDE       0.75      0.80      0.77       908

   micro avg       0.74      0.74      0.74      2616
   macro avg       0.74      0.74      0.74      2616
weighted avg       0.74      0.74      0.74      2616

0.7412079510703364


# KNN

In [9]:
from sklearn.neighbors import KNeighborsClassifier
# fit a k-nearest neighbor model to the data
knn = KNeighborsClassifier()
y_pred = knn.fit(X_train, y_train).predict(X_test)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[489 177  69]
 [165 669 139]
 [112 122 674]]
              precision    recall  f1-score   support

      INSIDE       0.64      0.67      0.65       735
IN_VESTIBULE       0.69      0.69      0.69       973
     OUTSIDE       0.76      0.74      0.75       908

   micro avg       0.70      0.70      0.70      2616
   macro avg       0.70      0.70      0.70      2616
weighted avg       0.70      0.70      0.70      2616

0.7003058103975535


# Добыча нового признака путем вычетания rssiOne - rssiOne

In [10]:
rssiOne = list()
c = 0
for index, row in raw_dataset.iterrows():
    if index == 0:
        difference = 0.0
    else:
#         print(c)
        if(c < 8717):
            difference = (row['rssiOne'] - raw_dataset.iloc[c+1, 5])
    rssiOne.append(difference)
    c += 1
rssiOne = np.array(rssiOne)
raw_dataset['rssiOne-rssiOne'] = pd.Series(rssiOne, index = raw_dataset.index)

In [11]:
raw_dataset.head()

,_id,name,locationStatus,direction,timestamp,rssiOne,rssiTwo,rssiOne-rssiOne
0,4744423E612A1ED103904DB4F5987BA6,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367500637,-79,-90,0.0
1,C245BE3DABC6B3B534FD3D0E660049B2,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367501323,-81,-90,2.0
2,854BB66FF73F63F0FB15149B80434D80,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367502416,-83,-91,-3.0
3,1E448462718ADB1DD73E9553A7B268B7,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367504813,-80,-91,0.0
4,0D4BAE97FE52DBAF3E596C72CC45C78F,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367504813,-80,-91,-3.0


In [12]:
clean_dataset = raw_dataset.drop(["_id"], axis=1)
show_df(clean_dataset)

(8718, 7)

,name,locationStatus,direction,timestamp,rssiOne,rssiTwo,rssiOne-rssiOne
0,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367500637,-79,-90,0.0
1,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367501323,-81,-90,2.0
2,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367502416,-83,-91,-3.0
3,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367504813,-80,-91,0.0
4,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367504813,-80,-91,-3.0


In [13]:
x = clean_dataset.drop(["locationStatus", "timestamp", "name"], axis=1)
y = clean_dataset["locationStatus"]

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.3, stratify=y)
X_train.head()

,direction,rssiOne,rssiTwo,rssiOne-rssiOne
1817,1,-66,-91,10.0
5379,0,-89,-79,-4.0
9477,1,-88,-86,7.0
2777,0,-84,-91,1.0
9925,0,-84,-79,1.0


In [14]:
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[459 107 169]
 [ 90 712 171]
 [107 113 688]]
              precision    recall  f1-score   support

      INSIDE       0.70      0.62      0.66       735
IN_VESTIBULE       0.76      0.73      0.75       973
     OUTSIDE       0.67      0.76      0.71       908

   micro avg       0.71      0.71      0.71      2616
   macro avg       0.71      0.70      0.71      2616
weighted avg       0.71      0.71      0.71      2616

0.7106269113149847


In [15]:
clf = tree.DecisionTreeClassifier()
y_pred = clf.fit(X_train, y_train).predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[555  90  90]
 [125 694 154]
 [105 181 622]]
              precision    recall  f1-score   support

      INSIDE       0.71      0.76      0.73       735
IN_VESTIBULE       0.72      0.71      0.72       973
     OUTSIDE       0.72      0.69      0.70       908

   micro avg       0.72      0.72      0.72      2616
   macro avg       0.71      0.72      0.72      2616
weighted avg       0.72      0.72      0.71      2616

0.7152140672782875


In [16]:
svm = SVC()
y_pred = svm.fit(X_train, y_train).predict(X_test)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

C:\anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[[574  72  89]
 [ 85 752 136]
 [ 70 110 728]]
              precision    recall  f1-score   support

      INSIDE       0.79      0.78      0.78       735
IN_VESTIBULE       0.81      0.77      0.79       973
     OUTSIDE       0.76      0.80      0.78       908

   micro avg       0.79      0.79      0.79      2616
   macro avg       0.79      0.79      0.79      2616
weighted avg       0.79      0.79      0.79      2616

0.7851681957186545


In [17]:
knn = KNeighborsClassifier()
y_pred = knn.fit(X_train, y_train).predict(X_test)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[589  63  83]
 [ 98 724 151]
 [ 99 115 694]]
              precision    recall  f1-score   support

      INSIDE       0.75      0.80      0.77       735
IN_VESTIBULE       0.80      0.74      0.77       973
     OUTSIDE       0.75      0.76      0.76       908

   micro avg       0.77      0.77      0.77      2616
   macro avg       0.77      0.77      0.77      2616
weighted avg       0.77      0.77      0.77      2616

0.7672018348623854


# Добыча признака путем вычетания rssiTwo - rssiTwo

In [19]:
rssiTwo = list()
c = 0
for index, row in raw_dataset.iterrows():
    if index == 0:
        difference = 0.0
    else:
        if(c < 8717):
            difference = (row['rssiTwo'] - raw_dataset.iloc[c+1, 6])
    rssiTwo.append(difference)
    c += 1
rssiTwo = np.array(rssiTwo)
raw_dataset['rssiTwo-rssiTwoe'] = pd.Series(rssiTwo, index = raw_dataset.index)

In [23]:
raw_dataset.head()

,_id,name,locationStatus,direction,timestamp,rssiOne,rssiTwo,rssiOne-rssiOne,rssiTwo-rssiTwoe
0,4744423E612A1ED103904DB4F5987BA6,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367500637,-79,-90,0.0,0.0
1,C245BE3DABC6B3B534FD3D0E660049B2,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367501323,-81,-90,2.0,1.0
2,854BB66FF73F63F0FB15149B80434D80,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367502416,-83,-91,-3.0,0.0
3,1E448462718ADB1DD73E9553A7B268B7,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367504813,-80,-91,0.0,0.0
4,0D4BAE97FE52DBAF3E596C72CC45C78F,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367504813,-80,-91,-3.0,-1.0


In [22]:
clean_dataset = raw_dataset.drop(["_id"], axis=1)
show_df(clean_dataset)

(8718, 8)

,name,locationStatus,direction,timestamp,rssiOne,rssiTwo,rssiOne-rssiOne,rssiTwo-rssiTwoe
0,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367500637,-79,-90,0.0,0.0
1,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367501323,-81,-90,2.0,1.0
2,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367502416,-83,-91,-3.0,0.0
3,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367504813,-80,-91,0.0,0.0
4,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367504813,-80,-91,-3.0,-1.0


In [24]:
x = clean_dataset.drop(["locationStatus", "timestamp", "name"], axis=1)
y = clean_dataset["locationStatus"]

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.3, stratify=y)
X_train.head()

,direction,rssiOne,rssiTwo,rssiOne-rssiOne,rssiTwo-rssiTwoe
339,0,-85,-92,8.0,0.0
6581,0,-79,-80,2.0,-2.0
5087,0,-82,-86,-8.0,-12.0
1759,1,-71,-88,3.0,-3.0
3678,1,-79,-71,-6.0,3.0


In [25]:
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[498 103 134]
 [111 737 125]
 [108 113 687]]
              precision    recall  f1-score   support

      INSIDE       0.69      0.68      0.69       735
IN_VESTIBULE       0.77      0.76      0.77       973
     OUTSIDE       0.73      0.76      0.74       908

   micro avg       0.73      0.73      0.73      2616
   macro avg       0.73      0.73      0.73      2616
weighted avg       0.73      0.73      0.73      2616

0.7347094801223242


In [26]:
clf = tree.DecisionTreeClassifier()
y_pred = clf.fit(X_train, y_train).predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[545 127  63]
 [131 737 105]
 [ 71  93 744]]
              precision    recall  f1-score   support

      INSIDE       0.73      0.74      0.74       735
IN_VESTIBULE       0.77      0.76      0.76       973
     OUTSIDE       0.82      0.82      0.82       908

   micro avg       0.77      0.77      0.77      2616
   macro avg       0.77      0.77      0.77      2616
weighted avg       0.77      0.77      0.77      2616

0.7744648318042814


In [27]:
svm = SVC()
y_pred = svm.fit(X_train, y_train).predict(X_test)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

C:\anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[[535 151  49]
 [ 62 858  53]
 [ 45 122 741]]
              precision    recall  f1-score   support

      INSIDE       0.83      0.73      0.78       735
IN_VESTIBULE       0.76      0.88      0.82       973
     OUTSIDE       0.88      0.82      0.85       908

   micro avg       0.82      0.82      0.82      2616
   macro avg       0.82      0.81      0.81      2616
weighted avg       0.82      0.82      0.82      2616

0.8157492354740061


In [28]:
knn = KNeighborsClassifier()
y_pred = knn.fit(X_train, y_train).predict(X_test)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[600  81  54]
 [ 99 805  69]
 [ 50  66 792]]
              precision    recall  f1-score   support

      INSIDE       0.80      0.82      0.81       735
IN_VESTIBULE       0.85      0.83      0.84       973
     OUTSIDE       0.87      0.87      0.87       908

   micro avg       0.84      0.84      0.84      2616
   macro avg       0.84      0.84      0.84      2616
weighted avg       0.84      0.84      0.84      2616

0.8398318042813455


# rssiOne - rssiTwo

In [29]:
rssiOneTwo = list()
c = 0
for index, row in raw_dataset.iterrows():
    if index == 0:
        difference = 0.0
    else:
        if(c < 8717):
            difference = (row['rssiOne'] - row['rssiTwo'])
    rssiOneTwo.append(difference)
    c += 1
rssiOneTwo = np.array(rssiOneTwo)
raw_dataset['rssiOne-rssiTwo'] = pd.Series(rssiOneTwo, index = raw_dataset.index)

In [30]:
raw_dataset.head()

,_id,name,locationStatus,direction,timestamp,rssiOne,rssiTwo,rssiOne-rssiOne,rssiTwo-rssiTwoe,rssiOne-rssiTwo
0,4744423E612A1ED103904DB4F5987BA6,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367500637,-79,-90,0.0,0.0,0.0
1,C245BE3DABC6B3B534FD3D0E660049B2,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367501323,-81,-90,2.0,1.0,9.0
2,854BB66FF73F63F0FB15149B80434D80,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367502416,-83,-91,-3.0,0.0,8.0
3,1E448462718ADB1DD73E9553A7B268B7,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367504813,-80,-91,0.0,0.0,11.0
4,0D4BAE97FE52DBAF3E596C72CC45C78F,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367504813,-80,-91,-3.0,-1.0,11.0


In [31]:
clean_dataset = raw_dataset.drop(["_id"], axis=1)
show_df(clean_dataset)

(8718, 9)

,name,locationStatus,direction,timestamp,rssiOne,rssiTwo,rssiOne-rssiOne,rssiTwo-rssiTwoe,rssiOne-rssiTwo
0,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367500637,-79,-90,0.0,0.0,0.0
1,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367501323,-81,-90,2.0,1.0,9.0
2,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367502416,-83,-91,-3.0,0.0,8.0
3,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367504813,-80,-91,0.0,0.0,11.0
4,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367504813,-80,-91,-3.0,-1.0,11.0


In [32]:
x = clean_dataset.drop(["locationStatus", "timestamp", "name"], axis=1)
y = clean_dataset["locationStatus"]

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.3, stratify=y)
X_train.head()

,direction,rssiOne,rssiTwo,rssiOne-rssiOne,rssiTwo-rssiTwoe,rssiOne-rssiTwo
2234,0,-66,-77,19.0,8.0,11.0
9301,1,-83,-75,-3.0,7.0,-8.0
2124,0,-87,-91,-15.0,-5.0,4.0
1394,0,-80,-93,-4.0,-6.0,13.0
5744,1,-77,-74,5.0,9.0,-3.0


In [33]:
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[524 123  88]
 [119 680 174]
 [ 89  55 764]]
              precision    recall  f1-score   support

      INSIDE       0.72      0.71      0.71       735
IN_VESTIBULE       0.79      0.70      0.74       973
     OUTSIDE       0.74      0.84      0.79       908

   micro avg       0.75      0.75      0.75      2616
   macro avg       0.75      0.75      0.75      2616
weighted avg       0.75      0.75      0.75      2616

0.7522935779816514


In [34]:
clf = tree.DecisionTreeClassifier()
y_pred = clf.fit(X_train, y_train).predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[578  97  60]
 [132 749  92]
 [ 78  74 756]]
              precision    recall  f1-score   support

      INSIDE       0.73      0.79      0.76       735
IN_VESTIBULE       0.81      0.77      0.79       973
     OUTSIDE       0.83      0.83      0.83       908

   micro avg       0.80      0.80      0.80      2616
   macro avg       0.79      0.80      0.79      2616
weighted avg       0.80      0.80      0.80      2616

0.7962538226299695


In [35]:
svm = SVC()
y_pred = svm.fit(X_train, y_train).predict(X_test)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

C:\anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[[533 164  38]
 [ 51 862  60]
 [ 47 107 754]]
              precision    recall  f1-score   support

      INSIDE       0.84      0.73      0.78       735
IN_VESTIBULE       0.76      0.89      0.82       973
     OUTSIDE       0.88      0.83      0.86       908

   micro avg       0.82      0.82      0.82      2616
   macro avg       0.83      0.81      0.82      2616
weighted avg       0.83      0.82      0.82      2616

0.8214831804281345


In [36]:
knn = KNeighborsClassifier()
y_pred = knn.fit(X_train, y_train).predict(X_test)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[593 101  41]
 [ 82 818  73]
 [ 52  52 804]]
              precision    recall  f1-score   support

      INSIDE       0.82      0.81      0.81       735
IN_VESTIBULE       0.84      0.84      0.84       973
     OUTSIDE       0.88      0.89      0.88       908

   micro avg       0.85      0.85      0.85      2616
   macro avg       0.84      0.84      0.84      2616
weighted avg       0.85      0.85      0.85      2616

0.8467125382262997


# (rssiOne + rssiTwo) - ((rssiOne-rssiOne) + (rssiTwo-rssiTwo))

In [69]:
porridge = list()
c = 0
for index, row in raw_dataset.iterrows():
    if index == 0:
        difference = 0.0
    else:
        if(c < 8717):
            difference = ((row['rssiOne'] + row['rssiTwo']) - (raw_dataset.iloc[c, 7] + raw_dataset.iloc[c, 8]))
    porridge.append(difference)
    c += 1
porridge = np.array(porridge)
raw_dataset['porridge'] = pd.Series(porridge, index = raw_dataset.index)

In [70]:
raw_dataset.head()

,_id,name,locationStatus,direction,timestamp,rssiOne,rssiTwo,rssiOne-rssiOne,rssiTwo-rssiTwoe,rssiOne-rssiTwo,porridge
0,4744423E612A1ED103904DB4F5987BA6,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367500637,-79,-90,0.0,0.0,0.0,0.0
1,C245BE3DABC6B3B534FD3D0E660049B2,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367501323,-81,-90,2.0,1.0,9.0,-174.0
2,854BB66FF73F63F0FB15149B80434D80,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367502416,-83,-91,-3.0,0.0,8.0,-171.0
3,1E448462718ADB1DD73E9553A7B268B7,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367504813,-80,-91,0.0,0.0,11.0,-171.0
4,0D4BAE97FE52DBAF3E596C72CC45C78F,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367504813,-80,-91,-3.0,-1.0,11.0,-167.0


In [71]:
clean_dataset = raw_dataset.drop(["_id"], axis=1)
show_df(clean_dataset)

(8718, 10)

,name,locationStatus,direction,timestamp,rssiOne,rssiTwo,rssiOne-rssiOne,rssiTwo-rssiTwoe,rssiOne-rssiTwo,porridge
0,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367500637,-79,-90,0.0,0.0,0.0,0.0
1,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367501323,-81,-90,2.0,1.0,9.0,-174.0
2,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367502416,-83,-91,-3.0,0.0,8.0,-171.0
3,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367504813,-80,-91,0.0,0.0,11.0,-171.0
4,FF:FF:C2:1D:9B:54,OUTSIDE,0,1551367504813,-80,-91,-3.0,-1.0,11.0,-167.0


In [72]:
x = clean_dataset.drop(["locationStatus", "timestamp", "name"], axis=1)
y = clean_dataset["locationStatus"]

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.3, stratify=y)
X_train.head()

,direction,rssiOne,rssiTwo,rssiOne-rssiOne,rssiTwo-rssiTwoe,rssiOne-rssiTwo,porridge
3100,1,-83,-86,3.0,3.0,3.0,-175.0
7688,1,-95,-86,-15.0,-5.0,-9.0,-161.0
8342,0,-84,-85,6.0,1.0,1.0,-176.0
5810,1,-79,-79,6.0,-7.0,0.0,-157.0
4488,1,-77,-75,17.0,12.0,-2.0,-181.0


In [73]:
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[556  72 107]
 [ 62 779 132]
 [ 91  77 740]]
              precision    recall  f1-score   support

      INSIDE       0.78      0.76      0.77       735
IN_VESTIBULE       0.84      0.80      0.82       973
     OUTSIDE       0.76      0.81      0.78       908

   micro avg       0.79      0.79      0.79      2616
   macro avg       0.79      0.79      0.79      2616
weighted avg       0.79      0.79      0.79      2616

0.7931957186544343


In [74]:
clf = tree.DecisionTreeClassifier()
y_pred = clf.fit(X_train, y_train).predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[573  95  67]
 [108 777  88]
 [ 64 103 741]]
              precision    recall  f1-score   support

      INSIDE       0.77      0.78      0.77       735
IN_VESTIBULE       0.80      0.80      0.80       973
     OUTSIDE       0.83      0.82      0.82       908

   micro avg       0.80      0.80      0.80      2616
   macro avg       0.80      0.80      0.80      2616
weighted avg       0.80      0.80      0.80      2616

0.7993119266055045


In [75]:
svm = SVC()
y_pred = svm.fit(X_train, y_train).predict(X_test)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

C:\anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[[511 174  50]
 [ 49 865  59]
 [ 40 161 707]]
              precision    recall  f1-score   support

      INSIDE       0.85      0.70      0.77       735
IN_VESTIBULE       0.72      0.89      0.80       973
     OUTSIDE       0.87      0.78      0.82       908

   micro avg       0.80      0.80      0.80      2616
   macro avg       0.81      0.79      0.79      2616
weighted avg       0.81      0.80      0.80      2616

0.7962538226299695


In [76]:
knn = KNeighborsClassifier()
y_pred = knn.fit(X_train, y_train).predict(X_test)

# summarize the fit of the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[603  78  54]
 [ 84 804  85]
 [ 52  68 788]]
              precision    recall  f1-score   support

      INSIDE       0.82      0.82      0.82       735
IN_VESTIBULE       0.85      0.83      0.84       973
     OUTSIDE       0.85      0.87      0.86       908

   micro avg       0.84      0.84      0.84      2616
   macro avg       0.84      0.84      0.84      2616
weighted avg       0.84      0.84      0.84      2616

0.8390672782874617
